# Full Adder
Below is a Logisim diagram of a Full Adder circuit implementation. 
![](./images/full_adder_logisim.png)

First we begin by importing magma. We will use the prefix `m` to distinguish between magma functions and native Python.

In [1]:
import magma as m

The following information was taken from the Lattice data sheet describing the ice40 architecture. The `SB_LUT4` is a 4-input lookup table that we will configure to implement logic functions.
![](./images/SB_LUT4_diagram.png)
![](./images/SB_LUT4_verilog.png)

We can use `m.DeclareCircuit` to declare a circuit corresponding to the `SB_LUT4` primitive. We use `m.In` and `m.Out` to specify the directionality of a port. All the inputs and outputs are a single `m.Bit`. 

We also include the values `A0, A1, A2, A3` that enable us to conveniently specify the lookup table (LUT) initialization values. For example, to implement `I0 & I1 ^ I2 | I3`, we can compute the LUT initialization value as `A0 & A1 ^ A2 | A3`.

In [2]:
SB_LUT4 = m.DeclareCircuit("SB_LUT4",
                           "I0", m.In(m.Bit),
                           "I1", m.In(m.Bit),
                           "I2", m.In(m.Bit),
                           "I3", m.In(m.Bit),
                           "O",  m.Out(m.Bit))

A0 = 0xAAAA
A1 = 0xCCCC
A2 = 0xF0F0
A3 = 0xFF00

Based on our logisim diagram, we will need to define 3 logic gates:
* 2-bit and
* 3-bit or
* 3-bit xor

In [3]:
class And2(m.Circuit):
    name = "And2"
    IO = ["I0", m.In(m.Bit), "I1", m.In(m.Bit), "O", m.Out(m.Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0&A1, 16))
        m.wire(io.I0, lut.I0)
        m.wire(io.I1, lut.I1)
        m.wire(0, lut.I2)
        m.wire(0, lut.I3)
        m.wire(lut.O, io.O)

class Or3(m.Circuit):
    name = "Or3"
    IO = ["I0", m.In(m.Bit), "I1", m.In(m.Bit), "I2", m.In(m.Bit), "O", m.Out(m.Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0|A1|A3, 16))
        m.wire(io.I0, lut.I0)
        m.wire(io.I1, lut.I1)
        m.wire(io.I2, lut.I2)
        m.wire(0, lut.I3)
        m.wire(lut.O, io.O)


class Xor3(m.Circuit):
    name = "Xor3"
    IO = ["I0", m.In(m.Bit), "I1", m.In(m.Bit), "I2", m.In(m.Bit), "O", m.Out(m.Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0^A1^A3, 16))
        m.wire(io.I0, lut.I0)
        m.wire(io.I1, lut.I1)
        m.wire(io.I2, lut.I2)
        m.wire(0, lut.I3)
        m.wire(lut.O, io.O)

We can define standard Python functions to instance, wire up inputs, and return the output of our logic gates.

* `And2()` returns an instance of the And2 circuit
* `and2_instance(a, b)` wires `a` and `b` to the inputs of `and2_instance` and returns the output `O`

In [4]:
def and2(a, b):
    and2_instance = And2()
    return and2_instance(a, b)

def or3(a, b, c):
    return Or3()(a, b, c)

def xor3(a, b, c):
    return Xor3()(a, b, c)

Finally, we can define a circuit to implement a Full Adder using our previously defined logic gates.

In [5]:
class FullAdder(m.Circuit):
    name = "FullAdder"
    IO = ["a", m.In(m.Bit), "b", m.In(m.Bit), "cin", m.In(m.Bit), 
          "out", m.Out(m.Bit), "cout", m.Out(m.Bit)]
    @classmethod
    def definition(io):
        # Generate the sum
        out = xor3(io.a, io.b, io.cin)
        m.wire(out, io.out)
        # Generate the carry
        a_and_b = and2(io.a, io.b)
        b_and_cin = and2(io.b, io.cin)
        a_and_cin = and2(io.a, io.cin)
        cout = or3(a_and_b, b_and_cin, a_and_cin)
        m.wire(cout, io.cout)

We can import magma's verilog backend to view the verilog definition corresponding to our FullAdder circuit. Notice the logic gates defined in terms of the `SB_LUT4` primitive.

In [6]:
from magma.backend.verilog import compile as compile_verilog

print(compile_verilog(FullAdder))

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
module Xor3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9966)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input  I0, input  I1, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I0), .I1(I1), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hFFEE)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module FullAdder (input  a, input  b, input  cin, output  out, output  cout);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
Xor3 inst0 (.I0(a), .I1(b), .I2(cin), .O(inst0_O));
And2 inst1 (.I0(a), .I1(b), .O(inst1_O));
And2 inst2 (.I0(b), .I1(cin), .O(inst2_O));
And2 inst3 (.I0(a), .I1(cin), .O(inst3_O));
Or3 inst4 (.I0(inst1_O), .I1(

We will test our circuit on the icestick by wiring the outputs to the `D1` and `D2` leds. We begin by importing the `IceStick` module from `loam`. With an instance of the `IceStick`, we turn on the clock, D1, and D2 pins. We then define a `main` function by calling `icestick.main()`, instancing our FullAdder, and wiring up the ports.

In [7]:
from loam.boards.icestick import IceStick
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()

main = icestick.main()
adder = FullAdder()
m.wire(0, adder.a)
m.wire(1, adder.b)
m.wire(1, adder.cin)
m.wire(adder.out, main.D1)
m.wire(adder.cout, main.D2)

import mantle lattice ice40
import mantle lattice mantle40


We use magma's `m.compile` function to generate verilog and pcf files for the icestick

In [8]:
m.compile("build/ice_full_adder", main)

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
compiling main


Finally, we use the yosys, arachne-pnr, and the icestorm tools to flash our circuit onto the icestick

In [9]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_full_adder.blif' ice_full_adder.v
arachne-pnr -q -d 1k -o ice_full_adder.txt -p ice_full_adder.pcf ice_full_adder.blif
icepack ice_full_adder.txt ice_full_adder.bin

iceprog ice_full_adder.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
